In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
import lightgbm as lgb
import scipy
from scipy import sparse
from pandas.core.common import SettingWithCopyWarning
import scipy.stats as sp
import pandas as pd
import numpy as np
from collections import Counter
import warnings
import time
import sys
import random
import os
import gc
import datetime

In [ ]:
path = '../data/'
data_path = '../trainTestData/'
middle_path = '../model/'

train_y = pd.read_csv(path + 'age_train.csv', names=['uid', 'label'])
sub = pd.read_csv(path + 'age_test.csv', names=['uid'])

train_csr = sparse.load_npz(data_path + 'trainData15112.npz')
test_csr = sparse.load_npz(data_path + 'testData15112.npz')
train_y = train_y["label"].values
# train_csr = train_csr[:200]
# train_y = train_y['label'].values[:200]
# print(train_csr.shape, test_csr.shape)

lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='multiclass',
    metrics='multi_error',
    num_class=6,
    n_estimators=20000,
    learning_rate=0.1,
    num_leaves=512,
    max_depth=-1,
    subsample=0.95,
    colsample_bytree=0.5,
    subsample_freq=1,
    reg_alpha=1,
    reg_lambda=1,
    random_state=42,
    n_jobs=10
)

oof = np.zeros((train_csr.shape[0], 6))
sub_preds = np.zeros((test_csr.shape[0], 6))
skf = StratifiedKFold(n_splits=5, random_state=812, shuffle=True)
t = time.time()


for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    print('Fold {}'.format(index + 1))
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                  eval_set=[(train_csr[train_index], train_y[train_index]),
                            (train_csr[test_index], train_y[test_index])],
                  eval_names=['train', 'valid'],
                  early_stopping_rounds=200, verbose=10)

    oof[test_index] = lgb_model.predict_proba(train_csr[test_index], num_iteration=lgb_model.best_iteration_)
    sub_preds += lgb_model.predict_proba(test_csr, num_iteration=lgb_model.best_iteration_) / skf.n_splits
#     lgb_model.savetxt(middle_path+'model/lgb_zl'+str(index)+'_model.txt')
    joblib.dump(lgb_model, '../model/lgb_zl_15112_2'+str(index)+'_model.pkl')

print(oof.shape, train_y.shape)

cv_final = accuracy_score(train_y, np.argmax(oof, axis=1)+1)
print('\ncv acc:', cv_final)

sub['label'] = np.argmax(sub_preds, axis=1) + 1
# sub.to_csv(middle_path + 'sub_{}.csv'.format(cv_final), index=False)


oof = np.zeros((train_csr.shape[0], 6))
sub_preds = np.zeros((test_csr.shape[0], 6))
skf = StratifiedKFold(n_splits=5, random_state=812, shuffle=True)
t = time.time()


for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    print('Fold {}'.format(index + 1))
    lgb_model = joblib.load('../model/lgb_zl_15112_2'+str(index)+'_model.pkl')

    oof[test_index] = lgb_model.predict_proba(train_csr[test_index], num_iteration=lgb_model.best_iteration_)
    sub_preds += lgb_model.predict_proba(test_csr, num_iteration=lgb_model.best_iteration_) / skf.n_splits


cv_final = accuracy_score(train_y, np.argmax(oof, axis=1)+1)
print('\ncv acc:', cv_final)

np.savetxt('../processed/lgboost_val_15112.txt', oof, fmt='%s', delimiter=',', newline='\n')
np.savetxt('../processed/lgboost_test_15112.txt', sub_preds, fmt='%s', delimiter=',', newline='\n')

sub['label'] = np.argmax(sub_preds, axis=1) + 1
# sub.to_csv(middle_path + 'sub_{}.csv'.format(cv_final), index=False)